# In questo notebook ottengo i vettori SVO degli utenti

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm

In [2]:
id_utenti = []
tweet_totali = []
tweet_positivi = []
tweet_neutrali = []
tweet_negativi = []
tweet_covid_totali = []
sentiments = []
volumes = []
objectivities = []
vectors = []

In [3]:
r = open('Dati_utente-tweet_war.csv', 'r', encoding='utf-8')

In [4]:
for line in r:
    line = line.strip()
    id_riga, id_utente_corrente, tweet_totali_corrente, tweet_positivi_corrente, tweet_neutrali_corrente, tweet_negativi_corrente, tweet_covid_totali_corrente = line.split(",")
    id_utenti.append(id_utente_corrente)
    tweet_totali.append(tweet_totali_corrente)
    tweet_positivi.append(tweet_positivi_corrente)
    tweet_neutrali.append(tweet_neutrali_corrente)
    tweet_negativi.append(tweet_negativi_corrente)
    tweet_covid_totali.append(tweet_covid_totali_corrente)
    volume_corrente = int(tweet_covid_totali_corrente)/int(tweet_totali_corrente)
    volumes.append(volume_corrente)
    objectivity_corrente = int(tweet_neutrali_corrente)/(int(tweet_positivi_corrente)+int(tweet_neutrali_corrente)+int(tweet_negativi_corrente))
    objectivities.append(objectivity_corrente)
    try:
        temp = (int(tweet_positivi_corrente)-int(tweet_negativi_corrente))/(int(tweet_positivi_corrente)+int(tweet_negativi_corrente))
        sentiment_corrente = 1/(1+pow(10, -temp))
        sentiments.append(sentiment_corrente)
        vettore_corrente = np.array([0.6*sentiment_corrente, 0.3*volume_corrente, 0.1*objectivity_corrente])
        vectors.append(vettore_corrente)
    except:
        sentiments.append(0)
        vettore_corrente = np.array([0.6*sentiment_corrente, 0.3*volume_corrente, 0.1*objectivity_corrente])
        vectors.append(vettore_corrente)

In [5]:
r.close()

In [6]:
df = pd.DataFrame(list(zip(id_utenti, tweet_totali, tweet_positivi, tweet_neutrali, tweet_negativi, tweet_covid_totali, sentiments, volumes, objectivities, vectors)), columns=['id_utente', 'tweet_totali', 'tweet_positivi', 'tweet_neutrali', 'tweet_negativi', 'tweet_covid_totali', 'sentiment', 'volume', 'objectivity', 'svo_vector'])

In [1]:
#ometto il risultato perchè ci apparivano gli id degli utenti e questo va contro la politica di Twitter
df

In [8]:
edges = []

In [17]:
for index, row in df.iterrows():
    for other_index, other_row in df.iterrows():
        if row['id_utente']>other_row['id_utente']:
            cosine = np.dot(row['svo_vector'],other_row['svo_vector'])/(norm(row['svo_vector'])*norm(other_row['svo_vector']))
            if cosine>0.99:
                tupla=(row['id_utente'], other_row['id_utente'])
                edges.append(tupla)

In [18]:
len(edges)

5791500

In [19]:
with open('covid_edges_099.txt', 'w') as temp_file:
    for item in edges:
        temp_file.write("%s,%s\n" %(item[0], item[1]))

In [8]:
with open('svo_war_user_moresentiment_lessvolume.txt', 'w') as temp_file:
    for index, row in df.iterrows():
        s=row['sentiment']*0.6
        v=row['volume']*0.3
        o=row['objectivity']*0.1
        temp_file.write("%s,%f,%f,%f\n" %(row['id_utente'], s, v, o))